# LWP Algorithm

In [1]:
#import timeit
#start = timeit.default_timer()
# All the program statements

In [2]:
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth', -1)
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [3]:
from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):

    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

## Input values

#### Distance & Speed are tunable 

In [4]:
speed_check    = 4
distance_check = 3
quantile_check = 0.1
bit_prob_check = 0.4
hour1_check    = 7
hour2_check    = 20
# distance check variables to be consider for tag correction in LWP cleanup
dist_check = 3
inter_dist_check = 2



#1.take row data & correct data time format and select required columns

In [5]:
raw_data = pd.read_csv(r"D:\Telenor_GP_BAN\LWP_input\VCDR_LWP_Input_set2.csv")

#print(raw_data.head())

raw_data1 = raw_data[['msisdn','call_start_time','call_identification_number','Lat_First_CI_tech', 'Lon_First_CI_tech']]

raw_data1.call_start_time = pd.to_datetime(raw_data1['call_start_time'],format = '%d-%m-%Y %H:%M:%S')

#raw_data1.info()

# Splitting Date
raw_data1["check_in_date"] = raw_data1.call_start_time.dt.date
raw_data1["check_in_hr"]   = raw_data1.call_start_time.dt.hour
raw_data1["check_in_min"]  = raw_data1.call_start_time.dt.minute
raw_data1["check_in_day"]  = raw_data1.call_start_time.dt.day


In [6]:
raw_data1.call_start_time.min(), raw_data1.call_start_time.max()

(Timestamp('2017-06-08 00:00:00'), Timestamp('2017-06-23 23:54:00'))

In [7]:
#This need to tune as per taken time duration
raw_data1 = raw_data1[(raw_data1['call_start_time'] >= '2017-06-01 00:00:00') & 
                      (raw_data1['call_start_time'] < '2017-06-22 00:00:00')]

raw_data1.call_start_time.min(), raw_data1.call_start_time.max()

(Timestamp('2017-06-08 00:00:00'), Timestamp('2017-06-21 23:40:00'))

In [8]:
day_count=raw_data1.groupby(['msisdn'])['check_in_date'].nunique().reset_index()

user_with_3_week_data=day_count[day_count['check_in_date']==14] # toral no of days to be change here

raw_data1=pd.merge(raw_data1,user_with_3_week_data[['msisdn']],on=['msisdn'],how='inner')

raw_data1.msisdn.nunique()

### Need to remove data of Lat-Long missing 

In [12]:
raw_data1=raw_data1.dropna()
raw_data1.isnull().sum()

msisdn                        0
call_start_time               0
call_identification_number    0
Lat_First_CI_tech             0
Lon_First_CI_tech             0
check_in_date                 0
check_in_hr                   0
check_in_min                  0
check_in_day                  0
dtype: int64

In [13]:
raw_data1.shape

(12994, 9)

# Step 1 to 2: Creating 30 min bins, Imputing missing bins

In [14]:
# Half-hour Tag
raw_data1.loc[raw_data1["check_in_min"]<30,"bin_hr"]  ='0'
raw_data1.loc[raw_data1["check_in_min"]>=30,"bin_hr"] ='1'

# Half-hour Bin
raw_data1['bin_hr'] = raw_data1['check_in_hr'].astype(str)+"_"+raw_data1['bin_hr'].astype(str)

# Unique Key
raw_data1['key'] = raw_data1['msisdn'].astype(str) + "_" + \
                   raw_data1['check_in_day'].astype(str) + "_" + \
                   raw_data1['bin_hr'].astype(str)

In [16]:
import copy
df_imputed   = copy.copy(raw_data1)

list_imsi    = list((raw_data1.msisdn.unique()))
user_length  = len(list_imsi)
day_length   = raw_data1["check_in_day"].nunique()
hr_length    = 48
df_dumy      = pd.DataFrame(list_imsi,columns=['msisdn'])
hr           = list(range(0,24))
bin          = [0,1]
min_bin      = ['00','30']
hr_bin       = [str(i)+'_'+ str(j) for i in hr for j in bin]
hr_min       = [str(i)+':'+ str(j) for i in hr for j in min_bin]

# Repeating the msisdn for (24 x 2)hrs x 23 days
df_dumy  = df_dumy.loc[df_dumy.index.repeat(day_length*hr_length)].reset_index(drop=True)

hr_binDf = pd.DataFrame(hr_bin,columns = ['bin_hr_dumy'])
hr_min_Df = pd.DataFrame(hr_min,columns = ['hr_min_dumy'])
hr_binDf = pd.concat([hr_binDf]*(day_length*user_length), ignore_index=True)
hr_binDf = hr_binDf.reset_index()
hr_binDf1 = pd.concat([hr_min_Df]*(day_length*user_length), ignore_index=True)
hr_binDf1 = hr_binDf1.reset_index()
hr_binDf=pd.merge(hr_binDf,hr_binDf1,on=['index'],how='left')
hr_binDf.head(2)

,index,bin_hr_dumy,hr_min_dumy
0,0,0_0,0:00
1,1,0_1,0:30


In [17]:
day   = list(range(min(raw_data1["check_in_day"]),max(raw_data1["check_in_day"])+1))

dayDf = pd.DataFrame(day,columns=['day_dumy'])
dayDf = dayDf.loc[dayDf.index.repeat(hr_length)].reset_index(drop=True)
dayDf = pd.concat([dayDf]*(user_length), ignore_index=True)

df_dumy = df_dumy.join(hr_binDf)
df_dumy = df_dumy.join(dayDf)
df_dumy = df_dumy.sort_values(['msisdn','index','bin_hr_dumy'],ascending=[True,True,True])
df_dumy.head()

,msisdn,index,bin_hr_dumy,hr_min_dumy,day_dumy
10080,8081090326,10080,0_0,0:00,8
10081,8081090326,10081,0_1,0:30,8
10082,8081090326,10082,1_0,1:00,8
10083,8081090326,10083,1_1,1:30,8
10084,8081090326,10084,2_0,2:00,8


In [18]:
df_dumy['key'] = df_dumy['msisdn'].astype(str) + "_" + \
                 df_dumy['day_dumy'].astype(str) +  "_" + \
                 df_dumy['bin_hr_dumy'].astype(str)

df_imputed1 = pd.merge(df_dumy,df_imputed,on=['key'],how='left')
df_imputed1.head()

,msisdn_x,index,bin_hr_dumy,hr_min_dumy,day_dumy,key,msisdn_y,call_start_time,call_identification_number,Lat_First_CI_tech,Lon_First_CI_tech,check_in_date,check_in_hr,check_in_min,check_in_day,bin_hr
0,8081090326,10080,0_0,0:00,8,8081090326_8_0_0,nan,NaT,NaN,nan,nan,NaN,nan,nan,nan,NaN
1,8081090326,10081,0_1,0:30,8,8081090326_8_0_1,nan,NaT,NaN,nan,nan,NaN,nan,nan,nan,NaN
2,8081090326,10082,1_0,1:00,8,8081090326_8_1_0,nan,NaT,NaN,nan,nan,NaN,nan,nan,nan,NaN
3,8081090326,10083,1_1,1:30,8,8081090326_8_1_1,nan,NaT,NaN,nan,nan,NaN,nan,nan,nan,NaN
4,8081090326,10084,2_0,2:00,8,8081090326_8_2_0,nan,NaT,NaN,nan,nan,NaN,nan,nan,nan,NaN


In [19]:
df_imputed1['month']=6
df_imputed1['year']=2017
df_imputed1['Dumy_date_time']=df_imputed1['day_dumy'].astype(str) + "-" + df_imputed1['month'].astype(str)+ "-" +df_imputed1['year'].astype(str)+ " " +df_imputed1['hr_min_dumy'].astype(str) 

In [20]:
df_imputed1.Dumy_date_time = pd.to_datetime(df_imputed1['Dumy_date_time'],format = '%d-%m-%Y %H:%M')

In [21]:
#imputed_data_final1.to_csv("df_imputed_final.csv")

In [22]:
msisdn=df_imputed1.msisdn_x.unique()
imputed_result=pd.DataFrame()
for i in msisdn:
    df=df_imputed1[df_imputed1['msisdn_x']==i]
    #impute_lst = ['msisdn_y', 'call_start_time', 'call_identification_number', 'Lat_First_CI_tech', 'Lon_First_CI_tech', 
    #              'check_in_date', 'check_in_hr', 'check_in_min','check_in_day', 'week', 'bin_hr']
    impute_lst = ['msisdn_y', 'call_identification_number', 'Lat_First_CI_tech', 'Lon_First_CI_tech']
    for j in impute_lst:
        df[j].fillna(method='pad', inplace=True)
    imputed_result=pd.concat([imputed_result,df],axis=0)
        

In [23]:
imputed_data_final=imputed_result[['msisdn_x','bin_hr_dumy','call_start_time','call_identification_number','Lat_First_CI_tech','Lon_First_CI_tech','Dumy_date_time']]
imputed_data_final.shape

(27169, 7)

In [24]:
imputed_data_final1=imputed_data_final[imputed_data_final['call_start_time'].isnull()]
imputed_data_final2=imputed_data_final[imputed_data_final['call_start_time'].notnull()]

imputed_data_final1['call_start_time']=imputed_data_final1['Dumy_date_time']

imputed_data_final1=imputed_data_final2.append(imputed_data_final1)

imputed_data_final1.shape

In [28]:
imputed_data_final1=imputed_data_final1.iloc[:,0:-1]

In [29]:
imputed_data_final1.rename(columns={'msisdn_x':'msisdn'},inplace=True)
imputed_data_final1['day_dumy'] = pd.to_datetime(imputed_data_final1['call_start_time']).dt.day
imputed_data_final1['check_in_hr'] = pd.to_datetime(imputed_data_final1['call_start_time']).dt.hour
imputed_data_final1['check_in_min'] = pd.to_datetime(imputed_data_final1['call_start_time']).dt.minute
imputed_data_final1['check_in_date'] = pd.to_datetime(imputed_data_final1['call_start_time']).dt.date

In [31]:
df_imputed1=imputed_data_final1.copy()

### Step 3: Cleaning by taking first and last time stamp in each bin

In [32]:

min_hr = df_imputed1.sort_values(['check_in_hr','check_in_min'],
                                 ascending = [True,True]).groupby(['msisdn','day_dumy','bin_hr_dumy']).head(1)

max_hr = df_imputed1.sort_values(['check_in_hr','check_in_min'],
                                 ascending = [True,False]).groupby(['msisdn','day_dumy','bin_hr_dumy']).head(1)

final_df = pd.concat([min_hr,max_hr])
final_df.head(2)

,msisdn,bin_hr_dumy,call_start_time,call_identification_number,Lat_First_CI_tech,Lon_First_CI_tech,day_dumy,check_in_hr,check_in_min,check_in_date
26378,9971726973,0_0,2017-06-08,13874852,28.51,77.07,8,0,0,2017-06-08
0,8081090326,0_0,2017-06-08,NaN,nan,nan,8,0,0,2017-06-08


In [33]:
min_hr.shape, max_hr.shape, final_df.shape

((20160, 10), (20160, 10), (40320, 10))

In [34]:
final_df = final_df.drop_duplicates()
final_df.shape

(22860, 10)

In [35]:
#final_df.to_csv("final_df_v1.csv")

In [36]:
final_df1 = final_df[["msisdn","call_identification_number",
                      "Lat_First_CI_tech","Lon_First_CI_tech",
                      "bin_hr_dumy","day_dumy",'check_in_date',
                     "check_in_hr","check_in_min","call_start_time"]]

final_df1                 = final_df1.sort_values(['msisdn','day_dumy','call_start_time'], 
                                                  ascending=[True,True,True])
final_df1.reset_index(drop=True,inplace=True)
final_df2                 = final_df1
final_df2['Lat_shifted']  = final_df2.groupby('msisdn')['Lat_First_CI_tech'].transform(lambda x: x.shift())
final_df2['long_shifted'] = final_df2.groupby('msisdn')['Lon_First_CI_tech'].transform(lambda x: x.shift())
final_df2['time_taken']   = final_df2.groupby('msisdn')['call_start_time'].transform(lambda x: x-x.shift()).dt.seconds


### Step 4 & 5: Calculating Distance and Time Taken, and Aggregate the Data

In [37]:
final_df2['distance'] = final_df2.apply(lambda row : haversine(row['Lon_First_CI_tech'], row['Lat_First_CI_tech'],
                                                               row['long_shifted'], row['Lat_shifted']), axis = 1)

In [38]:
final_df2.to_csv(r"D:\Telenor_GP_BAN\Codes_output\distance_ouput_set2.csv")

In [39]:
final_df3 = final_df2.groupby(['msisdn','check_in_date', 
                               'day_dumy','check_in_hr',
                               'bin_hr_dumy'])['time_taken','distance'].sum().reset_index()

# If time taken is greater than 30 minutes (1800 seconds), then restrict to 1800 seconds
final_df3.loc[final_df3.time_taken >= 1800,'time_taken'] = 1800

# Calculating speed = distance/time
final_df3['speed'] = (final_df3['distance'] / final_df3['time_taken']) * 3600

In [40]:
final_df3.shape

(20160, 8)

In [41]:
#final_df3.to_csv("Step_5_op_v1.csv")

### Step 6: Tag Mobile and Stationary based on Condition

In [42]:
# If speed is greater than 4kmph and distance is greater than 3km, then assign status as Mobile
final_df3.loc[(final_df3.speed>speed_check) & (final_df3.distance>distance_check),'status'] = 'Mobile'

# If not, assign status as Stationary
final_df3.loc[final_df3.status.isnull(),'status']='Stationary'

In [43]:
final_df3.to_csv(r"D:\Telenor_GP_BAN\Codes_output\with_M_S_tag_set2.csv",index=False)

### Step 16: Filter rows where tag is 'Mobile'

In [44]:
# Filtering Mobile Tag
final_df4 = final_df3[final_df3['status']=='Mobile']

#final_df4.shape

# Distinct Weeks
final_df4.loc[(final_df4["day_dumy"]<=7),"week"]=int(1)
final_df4.loc[(final_df4["day_dumy"]> 7)  & (final_df4["day_dumy"]<=14),"week"] = int(2)
final_df4.loc[(final_df4["day_dumy"]> 14) & (final_df4["day_dumy"]<=21),"week"] = int(3)
final_df4.loc[(final_df4["day_dumy"]> 21) & (final_df4["day_dumy"]<=28),"week"] = int(4)
#final_df4['week'].unique()

final_df4.drop_duplicates(keep=False,inplace=True)

### Step 17: Aggregate the Data

In [48]:
final_df4['check_in_date'] = pd.to_datetime(final_df4['check_in_date'])
final_df4['dayofweek'] = final_df4['check_in_date'].dt.day_name()


# tagging sunday & saturdat as weekend & rest as weekday
def com(x):
    if((x['dayofweek']=='Sunday') or (x['dayofweek']=='Saturday')):
        return 'weekend'
    else:
        return 'weekday'
final_df4['weekday_weekend']= final_df4.apply(com,axis=1)

# filter weeday & weekend data 
weekday_data=final_df4[(final_df4['weekday_weekend']=='weekday')]
weekend_data=final_df4[(final_df4['weekday_weekend']=='weekend')]


In [49]:
weekday_data.dayofweek.value_counts()

Monday       327
Tuesday      324
Friday       287
Thursday     272
Wednesday    267
Name: dayofweek, dtype: int64

In [50]:
#total_sample=final_df4.groupby(['imsi','day_dumy','distance','time_taken','week']).agg(total_events=('day_dumy','count'),total_distance_travelled=('distance','sum'),total_time_spent=('time_taken','sum'),Distinct_week=('week','count')).reset_index()
total_sample = weekday_data.groupby(['msisdn','dayofweek','check_in_hr'
                                 ]).agg(total_events             = ('day_dumy','count'),
                                        total_distance_travelled = ('distance','sum'),
                                        total_time_spent         = ('time_taken','sum'),
                                        Distinct_week            = ('week','nunique')).reset_index()

df_imputed1['check_in_date'] = pd.to_datetime(df_imputed1['check_in_date'])
df_imputed1['dayofweek'] = df_imputed1['check_in_date'].dt.day_name()

Distinct_cells  =  df_imputed1.groupby(['msisdn','dayofweek','check_in_hr']).agg(Distinct_cells=('call_identification_number','nunique')).reset_index()
Distinct_cells.shape,total_sample.shape

In [53]:
Join_with_Distinct_cells=total_sample.merge(Distinct_cells,on=['msisdn','dayofweek','check_in_hr'],how='left')
Join_with_Distinct_cells.shape

(958, 8)

### Step 18: Calculate Mobility Factor

In [54]:
Join_with_Distinct_cells['Mobility_factor'] = Join_with_Distinct_cells['total_events']*\
                                  Join_with_Distinct_cells['total_distance_travelled']*\
                                  Join_with_Distinct_cells['total_time_spent']*\
                                  Join_with_Distinct_cells['Distinct_cells']
#Join_with_Distinct_cells.head()

Non_zero_sample = Join_with_Distinct_cells[Join_with_Distinct_cells['msisdn'].notnull()]
Non_zero_sample.shape,Join_with_Distinct_cells.shape

### Step 19: Mobility Factor "E" > "10th_perc_cut_off"

In [56]:
percentile_data    = Non_zero_sample.groupby(['msisdn'])['Mobility_factor'].quantile(quantile_check).reset_index()
percentile_data.rename(columns={'Mobility_factor':'10_percentile'},inplace=True)

percentile_cut_off = pd.merge(Join_with_Distinct_cells, percentile_data, on=(['msisdn']), how='left')
#percentile_cut_off.head()

### Step 20:  Mobility Factor calculation 
#### Note: Restriction 1(Easy) to reduce Mobility Hours --> MF>10th Percentile
#### Note: Restriction 2(Harsh) Cinsider Hours only if Distict Weeks > 2

In [57]:
#percentile_cut_off.loc[(percentile_cut_off['Mobility_factor']>percentile_cut_off['10_percentile']) & (percentile_cut_off['Distinct_week']>=2),'hourly_probability'] = 1
percentile_cut_off.loc[(percentile_cut_off['Mobility_factor']>percentile_cut_off['10_percentile']),'hourly_probability'] = 1

percentile_cut_off.hourly_probability.fillna(0,inplace=True)
#percentile_cut_off.head()
percentile_cut_off=percentile_cut_off.drop_duplicates()

### Step 23: Create 24 hours Mobility pattern for each user

In [60]:
user_pattren_24_7=percentile_cut_off.groupby(['msisdn','dayofweek','check_in_hr'])['hourly_probability'].sum().unstack(-1).fillna(0).reset_index()
user_pattren_24_7.replace({0:'S',1:"M",2:"M",3:"M",4:"M"},inplace=True)

In [61]:
percentile_cut_off.to_csv(r"D:\Telenor_GP_BAN\Codes_output\percentile_cut_off_set2.csv",index=False)

In [62]:
user_pattren_24_7.to_csv(r"D:\Telenor_GP_BAN\Codes_output\User_pattren_24x7_set2.csv",index=False)


# For All day 

In [63]:
# percentile_cut_off_agg=percentile_cut_off.groupby(['msisdn','check_in_hr']).agg(bit_sum=('hourly_probability','sum')).reset_index()

# percentile_cut_off_agg['bit_sum']=percentile_cut_off_agg['bit_sum']/7
# percentile_cut_off_agg['Bit']=  np.where(percentile_cut_off_agg['bit_sum']>0.428, "M","S")

# # Pivot data 
# all_day_pattren=percentile_cut_off_agg.groupby(['msisdn','check_in_hr'])['Bit'].sum().unstack(-1).fillna("S").reset_index()


In [64]:
# all_day_pattren.to_csv("all_day_pattren_v2.csv")

# For Weekday
#### Bit_Sum is tunable 

In [65]:
bit_prob_check

0.4

In [66]:
weekday_data_agg=percentile_cut_off.groupby(['msisdn','check_in_hr']).agg(bit_sum=('hourly_probability','sum')).reset_index()

weekday_data_agg['bit_sum']=weekday_data_agg['bit_sum']/5
weekday_data_agg['Bit']=  np.where(weekday_data_agg['bit_sum']>bit_prob_check, "M","S")

# Pivot datbit_prob_check
weekday_pattren=weekday_data_agg.groupby(['msisdn','check_in_hr'])['Bit'].sum().unstack(-1).fillna("S").reset_index()

In [67]:
weekday_pattren.to_csv(r"D:\Telenor_GP_BAN\Codes_output\Weekday_pattren_set2.csv",index=False)

# For Weekend

In [68]:
# weekend_data_agg=weekend_data.groupby(['msisdn','check_in_hr']).agg(bit_sum=('hourly_probability','sum')).reset_index()

# weekend_data_agg['bit_sum']=weekend_data_agg['bit_sum']/2
# weekend_data_agg['Bit']=  np.where(weekend_data_agg['bit_sum']>0.5, "M","S")

# # Pivot data 
# weekend_pattren=weekend_data_agg.groupby(['msisdn','check_in_hr'])['Bit'].sum().unstack(-1).fillna("S").reset_index()


In [69]:
# weekend_pattren.to_csv("Weekend_pattren.csv")

# Step 25

In [71]:
df_T = weekday_pattren.iloc[:,1:25].T

df_T.columns = weekday_pattren.msisdn
df_T.head(2)

msisdn,8081090326,8130512825,8376047165,8377946756,8800594342,8800894118,8826390220,8860080120,9560045522,9582023942,9582261362,9599522355,9654981192,9711300642,9711303913,9717285978,9717627838,9810040500,9810403756,9810462609,9818259866,9818783881,9829086697,9910334990,9910526500,9953467155,9953562426,9958176146,9971430165,9971726973
check_in_hr,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S
1,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S,S


In [73]:
# To convert S to H where no M exists

pattern_array = df_T.to_numpy()
for i in range(weekday_pattren.msisdn.nunique()):
    if pattern_array[:,i].tolist().count('M')<=1:
        pattern_array[:,i]='H'

df_T_0 = pd.DataFrame(pattern_array, columns = df_T.columns)
df_T_0.T.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
msisdn,,,,,,,,,,,,,,,,,,,,,,,,
8081090326,S,S,S,S,S,S,S,S,S,S,M,M,S,M,M,S,S,M,S,M,M,M,S,S
8130512825,S,S,S,S,S,S,S,S,S,S,M,M,M,M,M,S,S,S,M,S,S,S,S,S
8376047165,S,S,S,S,S,S,S,S,S,M,S,S,M,M,S,S,M,M,M,M,M,S,S,S
8377946756,S,S,S,S,S,S,S,S,S,S,S,M,S,S,S,S,S,S,S,S,S,M,S,S
8800594342,S,S,S,S,S,S,S,S,S,M,S,S,S,S,S,M,S,S,S,S,S,S,S,S


## Convert S into M if S is between two M
### can be tunable

In [74]:
# Convert S into M if S is between two Ms

for user in range(weekday_pattren.msisdn.nunique()):
    for hr in range(22):
        if ((df_T_0.iloc[hr,user]=='M') & (df_T_0.iloc[hr+2,user]=='M')):
            df_T_0.iloc[hr+1,user]='M'
            
df_T_0.T.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
msisdn,,,,,,,,,,,,,,,,,,,,,,,,
8081090326,S,S,S,S,S,S,S,S,S,S,M,M,M,M,M,S,S,M,M,M,M,M,S,S


# Convert S into W when S is between two M
### tunable to optimize Work Location 

In [75]:
# Convert S into W when S is between two Ms within 7AM to 

for user in range(weekday_pattren.msisdn.nunique()):

    first_instance = df_T_0.iloc[:,user].where(df_T_0.iloc[:,user].values=='M').first_valid_index() or 0


    last_instance = df_T_0.iloc[:,user].where(df_T_0.iloc[:,user].values=='M').last_valid_index() or 0

    print(i,first_instance,last_instance)

    diff = last_instance-first_instance

    #if diff>=5:
    df_T_0.iloc[first_instance:last_instance,user] = np.where((df_T_0.iloc[first_instance:last_instance,user]=='S'),'W', 
                                                              df_T_0.iloc[first_instance:last_instance,user])

df_T_0.T.head()

29 10 21
29 10 18
29 9 20
29 11 21
29 9 15
29 10 11
29 0 0
29 0 0
29 12 18
29 11 22
29 9 19
29 0 0
29 11 20
29 8 20
29 9 19
29 10 21
29 10 19
29 9 20
29 0 0
29 0 0
29 9 21
29 9 20
29 10 17
29 11 21
29 9 14
29 9 23
29 0 0
29 11 12
29 4 18
29 11 15


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
msisdn,,,,,,,,,,,,,,,,,,,,,,,,
8081090326,S,S,S,S,S,S,S,S,S,S,M,M,M,M,M,W,W,M,M,M,M,M,S,S
8130512825,S,S,S,S,S,S,S,S,S,S,M,M,M,M,M,W,W,W,M,S,S,S,S,S
8376047165,S,S,S,S,S,S,S,S,S,M,W,W,M,M,W,W,M,M,M,M,M,S,S,S
8377946756,S,S,S,S,S,S,S,S,S,S,S,M,W,W,W,W,W,W,W,W,W,M,S,S
8800594342,S,S,S,S,S,S,S,S,S,M,W,W,W,W,W,M,S,S,S,S,S,S,S,S


In [76]:
df_T_fin = df_T_0.replace('S', 'H')

final_result=df_T_fin.T.reset_index()

In [78]:
final_result.to_csv(r"D:\Telenor_GP_BAN\Codes_output\final_result_on_20_percemtile_set2.csv",index=False)

In [79]:
pattern = final_result.melt(id_vars=["msisdn"], 
         var_name="check_in_hr", 
         value_name="tag")

In [80]:
imputed_data_final1.columns

Index(['msisdn', 'bin_hr_dumy', 'call_start_time',
       'call_identification_number', 'Lat_First_CI_tech', 'Lon_First_CI_tech',
       'day_dumy', 'check_in_hr', 'check_in_min', 'check_in_date'],
      dtype='object')

In [81]:
df=pd.merge(imputed_data_final1,pattern,on=["msisdn",'check_in_hr'],how='left')

df["lat_Long"]=df['Lat_First_CI_tech'].astype(str)+"_"+df['Lon_First_CI_tech'].astype(str)

In [85]:
df.tag.value_counts()

H    15770
M     7678
W     3721
Name: tag, dtype: int64

## No of  Top Lat-Long of User to choose H/W location
### it is tunable 

In [86]:
a=df.msisdn.unique()
result=pd.DataFrame()
for i in a:
    df1=df[df['msisdn']==i]
    df2=df1.groupby(['msisdn','lat_Long','tag']).size().unstack(-1).reset_index()
    #df2=df1.pivot_table(index=['lat_Long'], columns=['tag'], values=['tag']).reset_index()
    dfH=df2.sort_values(['H'],ascending=[False]).head(2)
    dfH['tag']='Home'
    try:
        dfW=df2.sort_values(['W'],ascending=[False]).head(2)
        dfW['tag']='Work'
    except:
        pass
    try:  
        dfM=df2.sort_values(['M'],ascending=[False]).head(2)
        dfM['tag']='Mobile'
    except:
        pass
    result=pd.concat([result,dfH,dfW,dfM],axis=0)

In [87]:
result.to_csv(r"D:\Telenor_GP_BAN\Codes_output\user_home&Work_lat_long_set2.csv",index=False)

In [88]:
df.to_csv(r"D:\Telenor_GP_BAN\Codes_output\result_for_address_mapping_set2.csv",index=False)

### LWP clean-up

In [256]:
df_factor =  df.groupby(['msisdn',"Lat_First_CI_tech",'Lon_First_CI_tech', 'tag']).agg(hour_count=('check_in_hr','count'),
                                                    unique_days=('check_in_date','nunique')).reset_index(drop=False)

df_factor['unique_days'] = df_factor['unique_days'].apply(lambda x: float(x))
df_factor['hour_count'] = df_factor['hour_count'].apply(lambda x: float(x))

df_factor['days_sq'] = df_factor['unique_days']**2
df_factor['factor'] = df_factor['days_sq'] * df_factor['hour_count']

df_factor["Rank"] = df_factor.groupby(['msisdn','tag'])["factor"].rank("dense", ascending=False)
df_factor.sort_values("Rank", inplace = True) 
df_factor.head()

,msisdn,Lat_First_CI_tech,Lon_First_CI_tech,tag,hour_count,unique_days,days_sq,factor,Rank
724,9654981192,28.58,77.30,M,35.00,11.00,121.00,"4,235.00",1.00
196,8376047165,28.55,77.23,H,247.00,11.00,121.00,"29,887.00",1.00
1669,9953467155,28.41,77.36,H,140.00,12.00,144.00,"20,160.00",1.00
854,9711303913,28.57,77.19,M,124.00,14.00,196.00,"24,304.00",1.00
333,8800594342,28.58,77.05,M,43.00,10.00,100.00,"4,300.00",1.00


In [257]:
df_top6 = df_factor[(df_factor['Rank'].isin([1,2,3,4,5,6]))]

df_Home = df_top5[(df_top5['tag']== 'H')]
df_Work = df_top5[(df_top5['tag']== 'W')]
df_Mobile = df_top5[(df_top5['tag']== 'M')]

df_top6.sort_values(['msisdn',"Rank"], inplace = True)

df_first_H= df_top6[(df_top6['Rank']== 1) & (df_top6['tag']== 'H')]


df_first_H.rename(columns = {'Lat_First_CI_tech':'Lat_first_H','Lon_First_CI_tech':'Long_first_H'}, inplace = True) 
 

df_merged_T6= pd.merge(df_top6,df_first_H[['msisdn','Lat_first_H','Long_first_H']], how='left',
               left_on=['msisdn'],right_on=['msisdn'])

In [258]:
import math
from math import sin, cos, sqrt, atan2, radians

def deg_to_rad(dr):
    return (dr*math.pi)/180

# approximate radius of earth in km
R = 6373.0

In [259]:
#distance Calculation from H1 to all H1-H6, M1-M6 & W1-W6
df_merged_T6['lat1'] = deg_to_rad(df_merged_T6['Lat_First_CI_tech'])
df_merged_T6['long1'] = deg_to_rad(df_merged_T6['Lon_First_CI_tech'])
df_merged_T6['lat2'] = deg_to_rad(df_merged_T6['Lat_first_H'])
df_merged_T6['long2'] = deg_to_rad(df_merged_T6['Long_first_H'])

df_merged_T6['dlong'] = df_merged_T6['long2'] - df_merged_T6['long1']
df_merged_T6['dlat'] = df_merged_T6['lat2'] - df_merged_T6['lat1']

df_merged_T6['a'] = np.sin(df_merged_T6['dlat'] / 2)**2 + np.cos(df_merged_T6['lat1']) * np.cos(df_merged_T6['lat2']) * np.sin(df_merged_T6['dlong'] / 2)**2

df_merged_T6['c'] = 2 * np.arctan2(np.sqrt(df_merged_T6['a']), np.sqrt(1 - df_merged_T6['a']))

df_merged_T6['distance_H1'] = R * df_merged_T6['c']

In [260]:
# dist_check parameter can be tune to change the tag
dist_check = 3
df_merged_T6['New_tag'] = ' '
df_merged_T6.loc[(df_merged_T6['distance_H1'] < dist_check), 'New_tag'] = 'Home'
#df_merged_T6.loc[(df_Home['intra_work_dist'] == 0) & (df_Home['tag_y'] == 'W'), 'new_tag'] = 'Work'

df_HOME = df_merged_T6[(df_merged_T6['New_tag'] == 'Home')]
df_remaining_t6 = df_merged_T6[(df_merged_T6['New_tag'] != 'Home')]

df_remaining_t6["New_Rank"] = df_remaining_t6.groupby(['msisdn','tag'])["factor"].rank("dense", ascending=False)
df_remaining_t6.sort_values("New_Rank", inplace = True) 

df_first_W= df_remaining_t6[(df_remaining_t6['New_Rank']== 1) & (df_remaining_t6['tag']== 'W')]

df_first_W.rename(columns = {'Lat_First_CI_tech':'Lat_first_W','Lon_First_CI_tech':'Long_first_W'}, inplace = True)

df_remaining_t6.drop(['lat1','long1','lat2','long2','dlong','dlat','a','c'], axis=1, inplace= True)

df_remaining_t6= pd.merge(df_remaining_t6,df_first_W[['msisdn','Lat_first_W','Long_first_W']], how='left',
               left_on=['msisdn'],right_on=['msisdn'])

In [261]:
#distance Calculation from H1 to all H1-H6, M1-M6 & W1-W6
df_remaining_t6['lat1'] = deg_to_rad(df_remaining_t6['Lat_First_CI_tech'])
df_remaining_t6['long1'] = deg_to_rad(df_remaining_t6['Lon_First_CI_tech'])
df_remaining_t6['lat2'] = deg_to_rad(df_remaining_t6['Lat_first_W'])
df_remaining_t6['long2'] = deg_to_rad(df_remaining_t6['Long_first_W'])

df_remaining_t6['dlong'] = df_remaining_t6['long2'] - df_remaining_t6['long1']
df_remaining_t6['dlat'] = df_remaining_t6['lat2'] - df_remaining_t6['lat1']

df_remaining_t6['a'] = np.sin(df_remaining_t6['dlat'] / 2)**2 + np.cos(df_remaining_t6['lat1']) * np.cos(df_remaining_t6['lat2']) * np.sin(df_remaining_t6['dlong'] / 2)**2

df_remaining_t6['c'] = 2 * np.arctan2(np.sqrt(df_remaining_t6['a']), np.sqrt(1 - df_remaining_t6['a']))

df_remaining_t6['distance_W1'] = R * df_remaining_t6['c']

In [262]:
# dist_check parameter can be tune to change the tag
dist_check = 3
df_remaining_t6.loc[(df_remaining_t6['distance_W1'] < dist_check), 'New_tag'] = 'Work'

df_remaining_t6.loc[(df_remaining_t6['New_tag'] == ' '), 'New_tag'] = 'Mobile'

df_temp1 = pd.concat([df_HOME,df_remaining_t6], ignore_index=True, sort=False)

df_LWP_master = df_temp1[['msisdn','Lat_First_CI_tech','Lon_First_CI_tech','tag','factor','Rank','distance_H1','distance_W1','New_Rank','New_tag']]

In [263]:
df_LWP_master.to_csv(r"D:\Telenor_GP_BAN\Codes_output\updated_Master_LWP_sheet.csv",index=False)